In [1]:
import pandas as pd

In [4]:
import pickle

# embeddings.pkl 파일 불러오기
with open('/content/embeddings.pkl', 'rb') as f:
    df = pickle.load(f)

# 데이터 확인
print(df.head())

   Unnamed: 0                            book_name  \
0           0          The Highly Sensitive Person   
1           1  Why Has Nobody Told Me This Before?   
2           2                 The Midnight Library   
3           3                      Brave New World   
4           4                                 1984   

                                           summaries categories  \
0   is a self-assessment guide and how-to-live te...    science   
1   is a collection of a clinical psychologist’s ...    science   
2   tells the story of Nora, a depressed woman in...    science   
3   presents a futuristic society engineered perf...    science   
4   is the story of a man questioning the system ...    science   

                                           embedding  
0  [-0.027592381462454796, 0.02508273534476757, 0...  
1  [-0.010329793207347393, 0.0206208024173975, 0....  
2  [-0.01274832896888256, -0.017568137496709824, ...  
3  [-0.00034115940798074007, -0.00196639355272054...

In [5]:
df.shape

(5201, 5)

In [6]:
# book_name 컬럼에서 중복 제거
df = df.drop_duplicates(subset='book_name')

# 중복 제거 결과 확인
print(df.head())
print(f"중복 제거 후 데이터프레임 크기: {df.shape}")


   Unnamed: 0                            book_name  \
0           0          The Highly Sensitive Person   
1           1  Why Has Nobody Told Me This Before?   
2           2                 The Midnight Library   
3           3                      Brave New World   
4           4                                 1984   

                                           summaries categories  \
0   is a self-assessment guide and how-to-live te...    science   
1   is a collection of a clinical psychologist’s ...    science   
2   tells the story of Nora, a depressed woman in...    science   
3   presents a futuristic society engineered perf...    science   
4   is the story of a man questioning the system ...    science   

                                           embedding  
0  [-0.027592381462454796, 0.02508273534476757, 0...  
1  [-0.010329793207347393, 0.0206208024173975, 0....  
2  [-0.01274832896888256, -0.017568137496709824, ...  
3  [-0.00034115940798074007, -0.00196639355272054...

In [7]:
import numpy as np
import pandas as pd

#  결측치(NaN 또는 None) 확인
missing_embeddings = df['embedding'].isnull().sum()
print(f"결측치(NaN 또는 None) 임베딩 개수: {missing_embeddings}")

#  임베딩 값이 올바른지(리스트인지) 확인
invalid_embeddings = df['embedding'].apply(lambda x: not isinstance(x, list)).sum()
print(f"리스트 형태가 아닌 임베딩 개수: {invalid_embeddings}")

#  임베딩 벡터 길이(차원) 검사 (예: 1536차원)
incorrect_dim_embeddings = df['embedding'].apply(lambda x: isinstance(x, list) and len(x) != 1536).sum()
print(f"1536차원이 아닌 임베딩 개수: {incorrect_dim_embeddings}")


결측치(NaN 또는 None) 임베딩 개수: 1
리스트 형태가 아닌 임베딩 개수: 1
1536차원이 아닌 임베딩 개수: 0


In [8]:
#  결측치(NaN 또는 None)인 데이터 출력
print("결측치(NaN 또는 None) 데이터:")
print(df[df['embedding'].isnull()])

#  리스트 형태가 아닌 데이터 출력
print("리스트 형태가 아닌 데이터:")
print(df[df['embedding'].apply(lambda x: not isinstance(x, list))])


결측치(NaN 또는 None) 데이터:
     Unnamed: 0                     book_name summaries     categories  \
823         829  The Power Of Full Engagement       NaN  relationships   

    embedding  
823      None  
리스트 형태가 아닌 데이터:
     Unnamed: 0                     book_name summaries     categories  \
823         829  The Power Of Full Engagement       NaN  relationships   

    embedding  
823      None  


In [9]:
# summaries 컬럼에서 결측치(NaN 또는 None) 제거
df = df[df['summaries'].notnull()]

# embedding 컬럼에서도 결측치(NaN 또는 None) 제거
df = df[df['embedding'].notnull()]

# 결과 확인
print(f"남은 데이터 개수: {len(df)}")


남은 데이터 개수: 1230


In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 유사한 책 검색 함수
def search_similar_books(book_title, top_n=5):
    try:
        # 입력값 존재 여부 확인
        if book_title not in df['book_name'].values:
            print(f"'{book_title}' 제목의 책을 찾을 수 없습니다.")
            return None

        # 입력된 책 제목, 줄거리, 장르 출력
        book_info = df.loc[df['book_name'] == book_title, ['book_name', 'summaries', 'categories']].iloc[0]
        print(" 입력한 책 정보")
        print(f" 제목: {book_info['book_name']}")
        print(f" 줄거리: {book_info['summaries']}")
        print(f" 장르: {book_info['categories']}\n")

        # 입력된 책 제목과 일치하는 책의 임베딩 불러오기
        query_embedding = np.array(df.loc[df['book_name'] == book_title, 'embedding'].values[0])

        # 책 임베딩 불러오기 (numpy 배열로 변환)
        book_embeddings = np.array(df['embedding'].tolist())

        # 코사인 유사도 계산
        similarities = cosine_similarity([query_embedding], book_embeddings)[0]

        # 유사도가 높은 Top N 도서 인덱스 추출 (자기 자신 제외)
        top_indices = similarities.argsort()[-(top_n + 1):][::-1]
        top_indices = [idx for idx in top_indices if df.iloc[idx]['book_name'] != book_title][:top_n]

        # 추천 결과 반환 (책 제목, 줄거리, 카테고리)
        return df.iloc[top_indices][['book_name', 'summaries', 'categories']]

    except Exception as e:
        print(f"유사도 계산 중 오류 발생: {e}")
        return None


In [16]:

# 예시 실행
book_title_input = input("추천을 원하는 책 제목을 입력하세요: ")
recommended_books = search_similar_books(book_title_input, top_n=5)

# 결과 확인
if recommended_books is not None:
    print(recommended_books)


추천을 원하는 책 제목을 입력하세요: 1984
 입력한 책 정보
 제목: 1984
 줄거리:  is the story of a man questioning the system that keeps his futuristic but dystopian society afloat and the chaos that quickly ensues once he gives in to his natural curiosity and desire to be free.
 장르: science

                               book_name  \
561           The Picture of Dorian Gray   
3                        Brave New World   
4469                          Wonderland   
27              The Sovereign Individual   
103   Brief Answers To The Big Questions   

                                              summaries     categories  
561    tells the story of a young, beautiful man who...  relationships  
3      presents a futuristic society engineered perf...        science  
4469   shows you that much of societal and technolog...     technology  
27     jumps into the future and presents a new worl...        science  
103    tackles some of the universe’s biggest myster...        science  


In [14]:
# 결과 확인 (한 번만 전체 출력)
if recommended_books is not None:
    print(recommended_books.to_string(index=False))
# 결과 5개
# 도리언 그레이의 초상
# 멋진 신세계
# 원더랜드 스티브 존슨https://product.kyobobook.co.kr/detail/S000000695602
# https://en.wikipedia.org/wiki/The_Sovereign_Individual
# 스티븐 호킹 논픽션

                         book_name                                                                                                                                                                                                                                                                  summaries    categories
        The Picture of Dorian Gray                                                            tells the story of a young, beautiful man who trades his soul for eternal youth, then descends further and further into a moral abyss — until he discovers there is, after all, a price to pay for his actions. relationships
                   Brave New World                                                                                               presents a futuristic society engineered perfectly around capitalism and scientific efficiency, in which everyone is happy, conform, and content — but only at first glance.       science
                        Wonderland                  

In [17]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 유사한 책 검색 함수 (동일한 장르만 추천)
def search_similar_books(book_title, top_n=5):
    try:
        # 입력값 존재 여부 확인
        if book_title not in df['book_name'].values:
            print(f"'{book_title}' 제목의 책을 찾을 수 없습니다.")
            return None

        # 입력된 책의 장르 확인
        book_info = df.loc[df['book_name'] == book_title, ['book_name', 'summaries', 'categories']].iloc[0]
        book_genre = book_info['categories']

        print("입력한 책 정보")
        print(f"제목: {book_info['book_name']}")
        print(f"줄거리: {book_info['summaries']}")
        print(f"장르: {book_genre}\n")

        # 동일한 장르의 책만 필터링
        genre_filtered_df = df[df['categories'] == book_genre]

        # 입력된 책 제목과 일치하는 책의 임베딩 불러오기
        query_embedding = np.array(df.loc[df['book_name'] == book_title, 'embedding'].values[0])

        # 필터링된 책 임베딩 불러오기 (numpy 배열로 변환)
        book_embeddings = np.array(genre_filtered_df['embedding'].tolist())

        # 코사인 유사도 계산
        similarities = cosine_similarity([query_embedding], book_embeddings)[0]

        # 유사도가 높은 Top N 도서 인덱스 추출 (자기 자신 제외)
        top_indices = similarities.argsort()[-(top_n + 1):][::-1]
        top_indices = [idx for idx in top_indices if genre_filtered_df.iloc[idx]['book_name'] != book_title][:top_n]

        # 추천 결과 반환 (책 제목, 줄거리, 카테고리)
        return genre_filtered_df.iloc[top_indices][['book_name', 'summaries', 'categories']]

    except Exception as e:
        print(f"유사도 계산 중 오류 발생: {e}")
        return None


In [18]:

# 예시 실행
book_title_input = input("추천을 원하는 책 제목을 입력하세요: ")
recommended_books = search_similar_books(book_title_input, top_n=5)

# 결과 확인
if recommended_books is not None:
    print(recommended_books)


추천을 원하는 책 제목을 입력하세요: 1984
입력한 책 정보
제목: 1984
줄거리:  is the story of a man questioning the system that keeps his futuristic but dystopian society afloat and the chaos that quickly ensues once he gives in to his natural curiosity and desire to be free.
장르: science

                              book_name  \
3                       Brave New World   
27             The Sovereign Individual   
103  Brief Answers To The Big Questions   
95                     Doubt: A History   
83                    The Order Of Time   

                                             summaries categories  
3     presents a futuristic society engineered perf...    science  
27    jumps into the future and presents a new worl...    science  
103   tackles some of the universe’s biggest myster...    science  
95    is a fascinating look at the historical influ...    science  
83    expands your mind by shattering your commonly...    science  
